# Download Data from Kaggle and perform split:
The data comes in 2 files : train, validation and test.
Unfortunately, the split performed seems to seperate movies from sets that don't cross over
A merge of data and a split is performed according taking in account the temporality: the 20% latest rating from each user will go to test set

At end, new train/test files are generated in/Data/generated

In [1]:
!pwd

In [2]:
import pandas as pd
import kaggle as kg
import numpy as np
from sklearn.model_selection import train_test_split
import gc, sys
from tqdm import tqdm_notebook as tqdm

DATA_FOLDER='../data/'

In [3]:
api = kg.KaggleApi('~/.kaggle/kaggle.json')
api.authenticate()
api.dataset_download_files('shuyangli94/food-com-recipes-and-user-interactions', DATA_FOLDER,
                           quiet=False, unzip=True)

  1%|          | 3.00M/267M [00:00<00:09, 28.6MB/s]

100%|██████████| 267M/267M [00:06<00:00, 40.2MB/s] 


In [4]:
df_full_data = pd.concat([pd.read_csv(f'{DATA_FOLDER}interactions_{typ}.csv') 
                          for typ in ['train', 'validation', 'test']])

#convert date column to datetime 
df_full_data['date'] = pd.to_datetime(df_full_data['date'], format='%Y-%m-%d')

In [5]:
print('number of recipes : ', len(df_full_data['i'].unique()))
print('number of users : ', len(df_full_data['u'].unique()))

number of recipes :  178265
number of users :  25076


In [6]:
##### Approach consists in placing the last ratings given by each user (approx 20%) in the test set. 

# Start with empty dataframes
train_set = pd.DataFrame(columns=['user_id', 'recipe_id', 'date', 'rating', 'u', 'i'])
test_set = pd.DataFrame(columns=['user_id', 'recipe_id', 'date', 'rating', 'u', 'i'])

train_df_list = []
test_df_list = []

# Loop over users - add last ratings given by each user to the test set 
mylist = list(set(df_full_data.u))
for item in tqdm(mylist, 'iterating over users'): 
    subdata = df_full_data[df_full_data['u']==item].sort_values(by='date')  # define a new dataframe for each user
    limit = round(0.2 * len(subdata))
    head = subdata.head(len(subdata) - limit)
    tail = subdata.tail(limit)
    train_df_list.append(head.copy())
    test_df_list.append(tail.copy())
    # put the other part of the data with ratings N/A => ensure that train and test have the same dimension
    with pd.option_context('mode.chained_assignment', None):
        head['rating']= np.nan
        tail['rating']= np.nan
    train_df_list.append(tail)
    test_df_list.append(head)
 
print("Starting build of train/test datasets")
train_set = pd.concat(train_df_list)
test_set = pd.concat(test_df_list)
print('Build completed')

print('train set shape is :',train_set.shape, 
      'deleted ratings number is :',train_set['rating'].isnull().sum(),
     'ratio is :', train_set['rating'].isnull().sum()/len(train_set))
print('test set shape is :',test_set.shape, 'deleted ratings number is :',
      test_set['rating'].isnull().sum(),
     'ratio is :', test_set['rating'].isnull().sum()/len(test_set))
    


Starting build of train/test datasets
Build completed
train set shape is : (718379, 6) deleted ratings number is : 143862 ratio is : 0.20025919465908665
test set shape is : (718379, 6) deleted ratings number is : 574517 ratio is : 0.7997408053409134


In [7]:
#######  Filter sparse recipes and users (not many associated ratings)
min_recipe_ratings = 20
min_user_ratings = 5

# Filter sparse recipes
filter_recipes = (train_set.dropna()['i'].value_counts()>min_recipe_ratings)
filter_recipes = filter_recipes[filter_recipes].index.tolist() # list of sparse recipes

# Create new dataset where these recipes are filtered
train_set_filter = train_set[train_set['i'].isin(filter_recipes)]

# Filter sparse users
filter_users = (train_set_filter.dropna()['u'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Create again new dataset, where users are now filtered
# We have do to it sequentially otherwise some users have rated more than 10 recipes but they can all get deleted as each of them was not rated more than 10 times
train_set_filter = train_set_filter[train_set_filter['u'].isin(filter_users)]

# Every recipe is rated at least 5-6 times. Same for users (more obvious as filtered in 2nd position)
print('Ensure that the value is 0 :',(train_set_filter['i'].value_counts().values < min_user_ratings).astype(int).sum())

print('Nomber of records unfiltered for train :', len(train_set.dropna()))
print('Nomber of records filtered for train :', len(train_set_filter.dropna()))

Ensure that the value is 0 : 0
Nomber of records unfiltered for train : 574517
Nomber of records filtered for train : 124321


In [8]:
#apply same filters apply to train to test
test_set_filter = test_set[test_set['i'].isin(filter_recipes)]
test_set_filter = test_set_filter[test_set_filter['u'].isin(filter_users)]

# Every recipe is rated at least 5-6 times. Same for users (more obvious as filtered in 2nd position)
print('Ensure that the value is 0 :',(test_set_filter['i'].value_counts().values < min_user_ratings).astype(int).sum())

print('Nomber of records unfiltered for test :', len(test_set.dropna()))
print('Nomber of records filtered for test :', len(test_set_filter.dropna()))

Ensure that the value is 0 : 0
Nomber of records unfiltered for test : 143862
Nomber of records filtered for test : 27662


In [9]:
#print the minimum number of rating per recipes for train
print('the minimum rating per recipe in train is :',
      train_set_filter.dropna().groupby(['i'])['rating'].count().sort_values().head(1).values[0])

#print the minimum number of rating per user for train
print('the minimum rating per user in train is :',
      train_set_filter.dropna().groupby(['u'])['rating'].count().sort_values().head(1).values[0])

#print the minimum number of rating per recipes for test
print('the minimum rating per recipe in test is :',
      test_set_filter.dropna().groupby(['i'])['rating'].count().sort_values().head(1).values[0])

#print the minimum number of rating per user for test
print('the minimum rating per user in test is :',
      test_set_filter.dropna().groupby(['u'])['rating'].count().sort_values().head(1).values[0])

the minimum rating per recipe in train is : 8
the minimum rating per user in train is : 6
the minimum rating per recipe in test is : 1
the minimum rating per user in test is : 1


In [10]:
##### Create a user-movie matrix for this approach
train_matrix = train_set_filter.pivot_table(index='u', columns='i', values='rating', dropna=False)
print(f'Shape of train User-Movie-Matrix:\t{train_matrix.shape}')

test_matrix = test_set_filter.pivot_table(index='u', columns='i', values='rating',dropna=False)
print(f'Shape of test User-Movie-Matrix:\t{test_matrix.shape}')

Shape of train User-Movie-Matrix:	(6384, 3148)
Shape of test User-Movie-Matrix:	(6384, 3148)


Save the files in 'DATA_FOLDER/data/generated'

In [11]:
train_set_filter[['rating', 'u', 'i']].to_csv(f'{DATA_FOLDER}generated/interactions_train_alt.csv',index=False)
test_set_filter[['rating', 'u', 'i']].to_csv(f'{DATA_FOLDER}generated/interactions_test_alt.csv', index=False)

Ensure that re building matrix provide the same result (size)

In [12]:
train_matrix = pd.read_csv('../data/generated/interactions_train_alt.csv').pivot_table(
                            index='u', columns='i', values='rating', dropna=False)
print(f'Shape of train User-Movie-Matrix:\t{train_matrix.shape}')

Shape of train User-Movie-Matrix:	(6384, 3148)


#### Merging PP_recipes and RAW_recipes

Write them in /data/generated/

In [13]:
pp_recipes = pd.read_csv('../data/PP_recipes.csv')
raw_recipes = pd.read_csv('../data/RAW_recipes.csv')

new_recipes = pp_recipes.join(raw_recipes.set_index(['id']), on='id', how = 'left')

display(new_recipes.head(3))

new_recipes.to_csv(f'{DATA_FOLDER}generated/recipes.csv', index=False)

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]",aromatic basmati rice rice cooker,61,496803,2010-05-10,"['weeknight', 'time-to-make', 'course', 'main-...","[228.2, 2.0, 2.0, 8.0, 9.0, 1.0, 15.0]",6,"['rinse the rice in a fine strainer , then dra...",from the ultimate rice cooker cookbook. the a...,"['basmati rice', 'water', 'salt', 'cinnamon st...",5
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,...",pumpkin pie a la easy,55,229619,2005-11-25,"['60-minutes-or-less', 'time-to-make', 'course...","[249.4, 16.0, 92.0, 8.0, 11.0, 27.0, 11.0]",10,"['preheat oven to 350', 'combine flour , oats ...",this is a pampered chef recipe for their stone...,"['flour', 'oats', 'brown sugar', 'pecans', 'bu...",12
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696...",cheesy tomato soup with potatoes,25,621626,2008-07-07,"['30-minutes-or-less', 'time-to-make', 'course...","[351.3, 34.0, 15.0, 50.0, 25.0, 70.0, 8.0]",6,"['pour the broth & water into a large pot', 'a...",after modifying another recipe i came up with ...,"['chicken broth', 'water', 'salt', 'black pepp...",15


In [14]:
#show object in memory
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir()], key=lambda x: x[1], reverse=True)

[('new_recipes', 538071181),
 ('raw_recipes', 388410531),
 ('pp_recipes', 254454202),
 ('test_matrix', 160825760),
 ('train_matrix', 160825760),
 ('df_full_data', 40229256),
 ('test_set', 40229256),
 ('train_set', 40229256),
 ('test_set_filter', 8511080),
 ('train_set_filter', 8511080),
 ('test_df_list', 406504),
 ('train_df_list', 406504),
 ('mylist', 225800),
 ('filter_users', 51144),
 ('filter_recipes', 25256),
 ('_i6', 1674),
 ('_i7', 1314),
 ('_i9', 871),
 ('_i8', 604),
 ('_i10', 425),
 ('_i', 339),
 ('_i13', 339),
 ('_i4', 312),
 ('_i12', 285),
 ('_ii', 285),
 ('_i11', 271),
 ('_iii', 271),
 ('_i3', 257),
 ('Out', 248),
 ('_oh', 248),
 ('_i2', 238),
 ('In', 200),
 ('_ih', 200),
 ('_i5', 172),
 ('_i14', 167),
 ('head', 144),
 ('subdata', 144),
 ('tqdm', 144),
 ('train_test_split', 144),
 ('__doc__', 113),
 ('__builtin__', 88),
 ('__builtins__', 88),
 ('gc', 88),
 ('kg', 88),
 ('np', 88),
 ('pd', 88),
 ('sys', 88),
 ('_dh', 80),
 ('get_ipython', 72),
 ('api', 64),
 ('exit', 64),
 (